In [1]:
import pandas as pd
from datetime import datetime

In [2]:
fin_df = pd.read_csv('Financials.csv')

fin_df.columns = ['Date', 'PE Ratio', 'Stock', 'EPS', 'PB Ratio', 'ROE (%)', 'PS Ratio', 'Shares Outstanding (M)', 'Revenue (M)']
fin_columns = ['EPS', 'ROE (%)', 'Revenue (M)']
fin_df[fin_columns] = fin_df[fin_columns].apply(lambda x : x.str.replace('[^0-9.]', '', regex = True).astype(float))
fin_df['Date'] = pd.to_datetime(fin_df.Date, utc = True).dt.date

In [3]:
stock_df = pd.read_csv('Stock_Price.csv')
stock_df["Date"] = pd.to_datetime(stock_df.Date, utc = True).dt.date

In [4]:
info_df = pd.read_csv('info.csv')

In [5]:
final_df = info_df.merge(stock_df, on = 'Stock')
final_df  = final_df.merge(fin_df, on = ['Date', 'Stock'], how = 'left')
final_df = final_df.sort_values(['Stock', 'Date'])
# final_df[fin_df.columns] = final_df[fin_df.columns].ffill()

In [8]:
final_df.to_csv('data_warehouse.csv', index = False)
stock_df.to_csv('Stock_Price.csv', index = False)

In [9]:
news_df = pd.read_csv('News.csv')
news_df['providerPublishTime'] = pd.to_datetime(news_df['providerPublishTime'].apply(datetime.fromtimestamp), utc = True).dt.date
news_df.to_csv('News.csv', index = False)

In [13]:
final_df.columns.tolist()

['industryKey',
 'sectorKey',
 'Stock',
 'Date',
 'Close',
 'Volume',
 'Dividends',
 'Stock Splits',
 'PE Ratio',
 'EPS',
 'PB Ratio',
 'ROE (%)',
 'PS Ratio',
 'Shares Outstanding (M)',
 'Revenue (M)']

In [14]:
final_df.groupby('Stock')['Revenue (M)'].count()

Stock
AAPL    43
AMD     43
JPM     43
NVDA    41
WFC     43
Name: Revenue (M), dtype: int64